## Decorators
Decorators are a significant part of Python. Simply put, they are functions that modify the functionality of other functions. They help to make our code shorter and more Pythonic

In [11]:
def first_decorator(func):
    def wrap():
        print('before')
        func()
        print('after')
    return wrap

In [12]:
def test():
    """test function docs"""
    print('inside test')

In [13]:
test.__name__

'test'

In [14]:
help(test)

Help on function test in module __main__:

test()
    test function docs



In [15]:
decorated_func = first_decorator(test)

In [16]:
test()

inside test


In [17]:
decorated_func()

before
inside test
after


Instead of writing code like this `decorated_func = first_decorator(test)` we can use special python syntax with `@` symbol

In [18]:
@first_decorator
def test():
    """test function docs"""
    print('inside test')

In [19]:
test()

before
inside test
after


In [20]:
test.__name__ #  the overwrites and docstring of our function

'wrap'

In [21]:
test.__doc__

In [22]:
from functools import wraps

In [23]:
def first_decorator(func):
    @wraps(func) # не обов'язково до використання
    def wrap():
        print('before')
        func()
        print('after')
    return wrap

@first_decorator
def test():
    """test function docs"""
    print('inside test')

In [24]:
test()

before
inside test
after


In [25]:
test.__name__

'test'

In [26]:
test.__doc__

'test function docs'

Passing arguments and a return value

In [27]:
def logging_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"\t[LOG] \t calling {func.__name__}")
        res = func(*args, **kwargs)
        print(f"\t[LOG] called {func.__name__}")
        return res
    return wrapper

# logging_decorator(func)(args)

In [28]:
@logging_decorator
def multiply(number_a:int, number_b:int):
    """ multiplies two provided numbers """
    print(f"\t\t=== Inside the multiply function")
    return number_a * number_b

In [29]:
result = multiply(4, 5)
print(f"[result] \t {result}")

	[LOG] 	 calling multiply
		=== Inside the multiply function
	[LOG] called multiply
[result] 	 20


Multiple decorators

In [30]:
def logging_decorator_1(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG1] \t calling decorator 1")
    res = func(*args, **kwargs)
    print(f"[LOG1] called decorator 1")
    return res
  return wrapper

def logging_decorator_2(func):
  def wrapper(*args, **kwargs):
    print(f"[LOG2] \t calling decorator 2")
    res = func(*args, **kwargs)
    print(f"[LOG2] called decorator 2")
    return res
  return wrapper

In [31]:
@logging_decorator_1
@logging_decorator_2
def multiply(number_a:int, number_b:int):
  """ multiplies two provided numbers """
  print("\tmultiply function")
  return number_a * number_b

In [32]:
result = multiply(4, 5)
print(f"[result] \t {result}")

# def decorator(func):
#     def modyficator(parameters):
#         result = func(parameters)
#         return result
#     return modyficator

[LOG1] 	 calling decorator 1
[LOG2] 	 calling decorator 2
	multiply function
[LOG2] called decorator 2
[LOG1] called decorator 1
[result] 	 20


Passing arguments to decorators

In [33]:
def add_brake_log(size=2):
    def add_brake_log_dec(func):
        @wraps(func)
        def wrap(*args, **kwargs):
            for _ in range(size):
                print('_' * 80)
            func(*args, **kwargs)
            for _ in range(size):
                print('_' * 80)
        return wrap
    return add_brake_log_dec

# def decorator(decorator_parameter=default):
#     def func_getter(func):
#         def modyficator(parameters):
#             if decorator_parameter is one:
#                 func(parameters)
#                 and one functinality
#             elif decorator_parameter is second:
#                 func(parameters)
#                 and another functinality
#             return result
#         return modyficator
#     return func_getter

In [34]:
@add_brake_log(size=3)
def test():
    """test function docs"""
    print('inside test')

test()

________________________________________________________________________________
________________________________________________________________________________
________________________________________________________________________________
inside test
________________________________________________________________________________
________________________________________________________________________________
________________________________________________________________________________


Decorator classes that have state

In [43]:
class ExecutionCounter:

  def __init__(self, func):
    self.func = func
    self.call_count = 0
  def __call__(self, *args, **kwargs):
    self.call_count += 1
    print(f"Called {self.func.__name__} for the {self.call_count}th time")
    return self.func(*args, **kwargs)

In [44]:
@ExecutionCounter
def multiply(number_a:int, number_b:int):
  """ multiplies two provided numbers """
  return number_a * number_b

In [45]:
res1 = multiply(21, 2)
res2 = multiply(14, 5)
res3 = multiply(6, 9)
res4 = multiply(4, 2)
print(f"res: {res1, res2, res3, res4}")

Called multiply for the 1th time
Called multiply for the 2th time
Called multiply for the 3th time
Called multiply for the 4th time
res: (42, 70, 54, 8)


Interaction between decorator args and function args

In [46]:
def check_password(password):
  def outer_wrapper(func):
    def wrapper(*args, **kwargs):
      provided_pass = kwargs.get("password") or args[0]
      if provided_pass != password:
        print(provided_pass, password)
        raise ValueError("Incorrect password")

      res = func(*args, **kwargs)
      return res
    return wrapper
  return outer_wrapper

In [47]:
@check_password(password="mypassword")
def public_api_request(password):
  """ Prints some text if the provided password is correct """
  print("You can only see this if you provided the right answer")

In [48]:
try:
    public_api_request(password='mypassword')
except Exception as e:
    print(e)

try:
    public_api_request(password='wrongpassword')
except Exception as e:
    print(e)

You can only see this if you provided the right answer
wrongpassword mypassword
Incorrect password


Decorators in API
- Валідація JSON
- Кешування
- Задання шляху і методу запиту
- Модифікація параметрів запиту

№1: @lru_cache -> прискорює рекурсивні виклики

In [1]:
from functools import lru_cache
@lru_cache(maxsize=4)
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(6)
fibonacci(6)
fibonacci(6)
fibonacci(6)
fibonacci(6)

Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


8

In [42]:
@app.route("/grade", methods=["POST"])
def update_grade():
    json_data = request.get_json()
    if "student_id" not in json_data:
        abort(400)
    # Update database
    return "success!"

NameError: name 'app' is not defined

№2: @jit -> попередньо компілює код функції

In [ ]:
from numba import jit
import random

@jit(nopython=True)
def monte_carlo_pi(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

№3: @do_twice ->

In [2]:
from decorators import do_twice
@do_twice
def timerfunc():
    %timeit factorial(15)

ImportError: cannot import name 'do_twice' from 'decorators' (D:\Python\Beetroot\venv\Lib\site-packages\decorators\__init__.py)

№4: @count_calls -> декоратор можна використовувати для надання інформації про те, скільки разів функція використовується у програмі

In [ ]:
from decorators import count_calls
@count_calls
def function_example():
    print("Hello World!")
function_example()
function_example()
function_example()

№5: @dataclass -> з допомогою цього декоратора можна швидко написати загальні стандартні методи, які зазвичай зустрічаються в класах, що ми пишемо

In [4]:
from dataclasses import dataclass

@dataclass
class Food:
    name: str
    unit_price: float
    stock: int = 0

    def stock_value(self) -> float:
        return(self.stock * self.unit_price)

№6: @singleton ->

In [6]:
def singleton(cls):
    instances = {}
    def wrapper(*args, **kwargs):
        if cls not in instances:
          instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return wrapper
@singleton
class cls:
    def func(self):
        pass

In [7]:
c1 = cls()
c2 = cls()
c3 = cls()
c4 = cls()
print(c1, c2, c3, c4, sep='\n')

№7: @use_unit ->  Це може бути корисно для тих, хто не хоче додавати одиниці виміру до своїх даних, але все одно хоче, щоб люди знали, що це за одиниці

In [8]:
import functools
import pint

def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit

@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [9]:
print(average_speed(1000, 3000))

0.3333333333333333 meter / second


№9: @singledispatch

In [11]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
        if verbose:
            print("Let me just say,", end=" ")
        print(arg)
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)
@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

In [13]:
fun([1, 2, 3, 4, 5], True)
fun(4, True)

Enumerate this:
0 1
1 2
2 3
3 4
4 5
Strength in numbers, eh? 4


№10: @register